# Geocoding with Python

Maps academies from the IAD dataset.

TODO: write this up

In [1]:
import os
import tqdm
import time
import requests
import zipfile
import folium
from folium.plugins import MarkerCluster
import xml.etree.ElementTree as ET
from geopy.geocoders import Nominatim
from urllib.request import Request

In [2]:
def download_dataset(url, directory):
    download_fn = url.split('/')[-1]
    download_path = os.path.join(directory, download_fn)
    if not os.path.exists(download_path):
        headers = {'User-agent': 'bl-digischol'}
        r = requests.get(url, stream=True, headers=headers)
        total_length = int(r.headers.get('Content-Length'))
        total_size = (total_length/1024) + 1
        with open(download_path, 'wb') as f:
            for chunk in tqdm.tqdm(r.iter_content(chunk_size=1024), 
                                   total=total_size, 
                                   desc='Downloading', 
                                   unit='kb',
                                   unit_scale=True, 
                                   miniters=1): 
                if chunk:
                    f.write(chunk)

download_dataset('https://data.bl.uk/iad/iad-xml.zip', './data')

In [3]:
def extract_dataset(fn, directory):
    basename = os.path.splitext(fn)[-2]
    in_path = os.path.join(directory, fn)
    out_path = os.path.join(directory, basename)
    with zipfile.ZipFile(in_path) as archive:
        unextracted = [name for name in archive.namelist() 
                       if not os.path.exists(os.path.join(out_path, name))]
        if unextracted:
            for i in tqdm.tqdm(range(len(unextracted)), desc='Extracting', unit='file'):
                archive.extract(unextracted[i], path=out_path)

extract_dataset('iad-xml.zip', './data')

In [16]:
geolocator = Nominatim(user_agent='bl-digischol')
records_dir = './data/iad-xml/records/ItacAcademyItem'
locations = []

for xml_file in os.listdir(records_dir):
    path = os.path.join(records_dir, xml_file)
    with open(path) as f:
        tree = ET.parse(f)
        root = tree.getroot()
        city = root.find(".//*/CityItalianName").text
        print(city)
        location = geolocator.geocode(city)
        coordinates = (location.latitude, location.longitude)
        locations.append(coordinates)
        time.sleep(1)  # Comply with usage policy


Bologna


GeocoderServiceError: HTTP Error 429: Too Many Requests

In [15]:
map = folium.Map(location=[41.87, 12.56], zoom_start=6)

marker_cluster = MarkerCluster().add_to(map)

for location in locations:
    folium.Marker(
        location=location
    ).add_to(marker_cluster)

map.save(os.path.join('data', 'iad.html'))

map